In [ ]:
import importlib
import retinanalysis as ra
from retinanalysis import dedup as dd
importlib.reload(dd)
importlib.reload(ra)
import pandas as pd
import numpy as np
ra.settings.mea_config
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import os

In [ ]:
#start with a noise file
df_1 = ra.get_datasets_from_protocol_names('.SpatialNoise')
df_1[df_1['exp_name'] == '20250514C']

In [ ]:
#set up our stimulus block and analysis chunk
sb = ra.MEAStimBlock(df_1.at[177, 'exp_name'], df_1.at[177, 'datafile_name'])
ac = ra.AnalysisChunk(df_1.at[177, 'exp_name'], sb.nearest_noise_chunk)

In [ ]:
#load in kilosort files, and add them in together so that we can analyze 
#TODO: make this a function
# amps = np.load('/Volumes/data/data/sorted/20250514C/chunk1/kilosort2.5/amplitudes.npy')
amps = np.load(os.path.join(ra.NAS_DATA_DIR, '20250514C', 'chunk1', 'kilosort2.5', 'amplitudes.npy'))
# templates = np.load('/Volumes/data/data/sorted/20250514C/chunk1/kilosort2.5/spike_templates.npy')
templates = np.load(os.path.join(ra.NAS_DATA_DIR, '20250514C', 'chunk1', 'kilosort2.5', 'spike_templates.npy'))

templates_vision = templates + 1
amplitudes = np.vstack((np.squeeze(amps), np.squeeze(templates_vision)))

In [ ]:
#plot ei autocorrelation, and spatial map autocorr if available
dd_block = dd.DedupBlock(df_1.at[177, 'exp_name'], df_1.at[177, 'datafile_name'], chunk_name = 'chunk1', is_noise = True,
                         ei_method='full', ei_threshold=0.8, sm_threshold=0.8)
dd_block.plot_correlations();

In [ ]:
#if available, plot all RFs and the RFs for the problem clusters
dd_block.plotRFs_dedup();

In [ ]:
#generate dataframe for all pairwise metrics for above-threshold clusters (in either ei or sm category)
#extended_pairs is a set of tuples of transitively related cells in the above-threshold correlation pairs
summary_stats = dd_block.get_summary_stats(dd_block.dedup_sets['sm_only_pairs'])
summary_stats

In [ ]:
#if we wanted to, we could just plot everything! (if detailed=True, plots timecourse of top electrodes)
#TODO: do we want this to be the default behavior? probably not - pull out best pairs to visualize?
#TODO: check marginal histograms for gaussian violations? to check if bimodal / needs splitting?
extended_pairs = dd.generate_extended_pairings(dd_block.dedup_sets['sm_only_pairs'])
for pair in extended_pairs:
    dd_block.visualize_groups(pair);
    dd_block.plot_amplitude_histograms(pair);